# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# your code here
df = pd.read_csv('tic-tac-toe.csv')

In [3]:
df.head()


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [4]:
df['class'].value_counts()


True     626
False    332
Name: class, dtype: int64

In [5]:
df.describe()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
count,958,958,958,958,958,958,958,958,958,958
unique,3,3,3,3,3,3,3,3,3,2
top,x,x,x,x,x,x,x,x,x,True
freq,418,378,418,378,458,378,418,378,418,626


In [6]:
df.shape

(958, 10)

In [7]:
df.isnull().sum()

TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64

In [9]:
df['class'].unique()
df['n_class'] = df['class'].astype(int) #change boolean to 0 and 1


In [10]:
df = df.drop(columns=['class'])
df.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,n_class
0,x,x,x,x,o,o,x,o,o,1
1,x,x,x,x,o,o,o,x,o,1
2,x,x,x,x,o,o,o,o,x,1
3,x,x,x,x,o,o,o,b,b,1
4,x,x,x,x,o,o,b,o,b,1


In [10]:
##df_n = pd.get_dummies(df)
#df_n.head()

In [11]:
cols = df.columns
cols

Index(['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'n_class'], dtype='object')

In [12]:
encoder = OneHotEncoder(handle_unknown='ignore')
df_enc = pd.DataFrame(encoder.fit_transform(df[['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'n_class']]).toarray())

In [13]:
join_df = df.join(df_enc)
join_df

,TL,TM,TR,ML,MM,MR,BL,BM,BR,n_class,...,19,20,21,22,23,24,25,26,27,28
0,x,x,x,x,o,o,x,o,o,1,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,x,x,x,x,o,o,o,x,o,1,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,x,x,x,x,o,o,o,o,x,1,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,x,x,x,x,o,o,o,b,b,1,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,x,x,x,x,o,o,b,o,b,1,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
954,o,x,o,x,x,o,x,o,x,0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
955,o,x,o,x,o,x,x,o,x,0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
956,o,x,o,o,x,x,x,o,x,0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [14]:
join_df.drop(columns=['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'n_class'], inplace=True)

In [15]:
join_df

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
954,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
955,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
956,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [16]:
scaler = StandardScaler()
scaler

StandardScaler()

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [17]:
# your code here

#use sklearn, or take skip method in keras??
#(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()


X, y = tf.split(df_n, num_or_size_splits=2, axis=0)

X_train, X_test, y_test, y_train = train_test_split(X, y, test_size=0.25)

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([397, 101, 193, 246, 302, 252, 413, 419, 404, 476,  43, 255,   8,
       429, 260, 317, 330, 475, 402, 349, 361, 195,  99,  42, 464, 128,
       219, 159,  39, 341, 116, 371, 423,  52, 158, 238, 326, 157, 431,
       181, 230, 231, 234, 120, 163, 375, 327, 226, 392, 239, 457, 161,
       379, 192, 359, 166, 106, 154, 297, 292, 373, 387,  81, 104, 256,
        88,  98, 315, 346, 208,  75, 319,  82, 108,  58, 445,  13, 170,
       191, 321, 199, 478, 217,  51, 261, 307, 299, 442, 366, 164, 229,
       275, 211, 203, 139, 365, 105, 312, 363, 276, 381,   6, 152, 334,
       250,  18,  67,  76,  87, 452, 168, 368,   3, 398,  16, 408, 364,
       390, 294, 322, 357, 391, 296, 360, 323, 342, 403,  60, 461, 257,
        47, 210, 433, 149, 447, 126, 141, 370,  65, 236, 300, 389, 145,
       189, 153, 382,  41, 122,  30, 232, 237, 112,   4,  12,   9, 344

In [ ]:
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, 3, activation='relu',
                              input_shape=(479, 29)),
        tf.keras.layers.MaxPool2D(2),
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.activations.softmax(x, axis=-1)
        
        tf.keras.layers.MaxPool2D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(10)
    
])

In [ ]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

In [ ]:
model.summary()
tic_tac_play = model.predict(***) #is this the right way to save y model??

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
# your code here

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here